# Imports and configuration

In [13]:
import torch
from torch import nn
from torch.nn import functional as F
import numpy as np
from matplotlib import pyplot as plt
import time
import pandas as pd
import random
from dataclasses import dataclass, field

In [14]:
# Create a config file for the cup shuffling task
@dataclass
class MASTER_CONFIG:
    # Training
    seed: int = 1337
    batch_size: int = 32
    training_split: float = 0.8
    
    max_iters = 100
    eval_interval = 10
    learning_rate = 2e-5
    eval_iters = 50
    
    # Model parameters
    dtype: torch.dtype = torch.float32
    d_model: int = 128 # This is the size of the embedding
    l_max: int = 128 # Max sequence length
    n_heads: int = 8 # Number of heads in the multi-head attention
    n_layers: int = 8 # Number of layers in the transformer of MHA blocks
    dropout: float = 0.1 # Dropout rate
    
    # Data parameters
    n_cups: int = 3
    n_moves: int = 3
    n_samples: int = 1000

    # Tokenizer
    vocab: list[str] = field(default_factory = lambda: [])
    vocab_size: int = 0
    
    # Use CUDA or MPS if available else CPU
    if (torch.cuda.is_available()):
        device = torch.device("cuda")
        print("Using CUDA")
    elif (torch.backends.mps.is_available()):
        device = torch.device("mps")
        print("Using Apple Silicon MPS")
    else:
        device = torch.device("cpu")
        print("Using CPU")

Using Apple Silicon MPS


# Ball Shuffler

In [15]:
import random

def initial_ball_position(n=3):
    return random.randint(1, n)

def generate_shuffle_moves(n=3, num_moves=3):
    moves = []
    
    for _ in range(num_moves):
        # Randomly pick two different cups
        cup1, cup2 = random.sample(range(1, n + 1), 2)
        moves.append((cup1, cup2))
    
    return moves

def final_ball_position(initial_position, shuffle_moves):
    position = initial_position
    for move in shuffle_moves:
        # If the ball's current position matches one of the cups in the move, swap it.
        if position == move[0]:
            position = move[1]
        elif position == move[1]:
            position = move[0]
    
    return position

# Method for generating data and labels for batches.
# TODO: This can be done much better
def generate_batch(split: str, config=MASTER_CONFIG):
    B = config.batch_size
    T = config.l_max
    vocab_size = config.vocab_size
    
    # Generate the data
    data = generate_cup_data(config.n_cups, config.n_moves, config.n_samples)
    
    # Split the data into training and validation sets
    if split == "train":
        data = data[:int(config.training_split * len(data))]
    elif split == "val":
        data = data[int(config.training_split * len(data)):]
    else:
        raise ValueError("split must be either 'train' or 'val'")
    
    # Return input and output tensors
    inputs = torch.zeros((B, T), dtype=torch.long)
    outputs = torch.zeros((B, T), dtype=torch.long)
    
    

def generate_cup_shuffling_scenario(n=3, num_moves=3):
    # Generate initial ball position and shuffle moves
    initial_position = initial_ball_position(n)
    shuffle_moves = generate_shuffle_moves(n, num_moves)
    
    # Calculate the final ball position
    final_position = final_ball_position(initial_position, shuffle_moves)
    
    # Construct the input and output strings
    str = f"There are {n} cups\n"
    str += f"Ball is in cup {initial_position}\n"
    str += "\n".join([f"Switch cup {move[0]} and cup {move[1]}" for move in shuffle_moves])
    str += f"\nBall is now in cup {final_position}"
    
    return str

# Generate batches of data
def generate_cup_data(n_cups = 3, num_examples=1000, num_moves=range(1, 5)):
    data = []
    
    for _ in range(num_examples):
        moves = random.choice(range(num_examples))
        input = generate_cup_shuffling_scenario(n_cups, moves)

        # Finds the last cup number in the string
        output = input.split()[-1][:-5]
        output = f" cup {output}"
        
        data.append((input, output))
    
    return data

In [16]:
initial_position = initial_ball_position()
print(initial_position)

shuffle_moves = generate_shuffle_moves()
print(shuffle_moves)

final_position = final_ball_position(initial_position, shuffle_moves)
print(final_position)

scenario = generate_cup_shuffling_scenario()
print(scenario)

print(generate_cup_data(n_cups=3, num_moves=range(1,5), num_examples=1))

2
[(1, 2), (3, 1), (1, 2)]
3
There are 3 cups
Ball is in cup 2
Switch cup 2 and cup 1
Switch cup 3 and cup 1
Switch cup 2 and cup 3
Ball is now in cup 2
[('There are 3 cups\nBall is in cup 2\n\nBall is now in cup 2', ' cup ')]


# Tokenizer

In [17]:
# Tokenizer, we want BOS, EOS tokens
vocab = ['<BOS>', '<EOS>', '\n', 'Ball', 'There are', ' is now in', ' is in', 'Switch', ' and', ' cup', ' 1', ' 2', ' 3', ' 4', ' 5', ' 6', ' 7', ' 8', ' 9', ' cups']

setattr(MASTER_CONFIG, "vocab", vocab)
setattr(MASTER_CONFIG, "vocab_size", len(vocab))

idx_to_s = {i:ch for i, ch in enumerate(vocab)}
s_to_idx = {ch:i for i, ch in enumerate(vocab)}

def encode_tokens(s: str) -> list[int]:
    ids = []
    i = 0
    while i < len(s):
        max_len = -1
        max_token = None
        for token in s_to_idx.keys():
            token_len = len(token)
            if s[i:i+token_len] == token:
                if token_len > max_len:
                    max_len = token_len
                    max_token = token
        if max_token:
            ids.append(s_to_idx[max_token])
            i += max_len
        else:
            print(f"Unrecognized sequence at index {i}, {s[i:i+1]}")
            
            break
    return ids

def decode_tokens(ids: list[int]) -> str:
    return "".join([idx_to_s[i] for i in ids])

s = generate_cup_shuffling_scenario(4, 5)

print(f"The string is {len(encode_tokens(s))} tokens long:\n\n{s}\n\nAnd has the following tokenization:\n{encode_tokens(s)}")


The string is 48 tokens long:

There are 4 cups
Ball is in cup 3
Switch cup 4 and cup 1
Switch cup 2 and cup 4
Switch cup 4 and cup 2
Switch cup 3 and cup 4
Switch cup 1 and cup 2
Ball is now in cup 4

And has the following tokenization:
[4, 13, 19, 2, 3, 6, 9, 12, 2, 7, 9, 13, 8, 9, 10, 2, 7, 9, 11, 8, 9, 13, 2, 7, 9, 13, 8, 9, 11, 2, 7, 9, 12, 8, 9, 13, 2, 7, 9, 10, 8, 9, 11, 2, 3, 5, 9, 13]


# Models, functions and layers

In [18]:
class DummyModel(nn.Module):
    """
    Dummy model for testing purposes. It takes a sequence of tokens and returns a probability distribution over the vocabulary.
    
    Input: x a sequence of tokens of shape (B, T)
    Output: a probability distribution over the vocabulary of shape (B, T, vocab_size)
    Parameters: 
    """
    def __init__(self, config=MASTER_CONFIG):
        super().__init__()
        self.config = config
        
        vocab_size = config.vocab_size
        d_model = config.d_model
        
        # Embedding layer from the vocabulary to the d_model dimension, parameter matrix W_e in R^{vocab_size x d_model}
        self.embedding = nn.Embedding(vocab_size, d_model)
        # Linear layer from d_model to vocab_size dimension. Parameter matrices W_l1, W_l2 in R^{d_model x d_model}, R^{d_model x vocab_size}
        self.linear = nn.Sequential(
            nn.Linear(d_model, d_model),
            nn.ReLU(),
            nn.Linear(d_model, vocab_size)
        )
        
        # Returns the number of parameters in the model
        self.num_parameters = sum(p.numel() for p in self.parameters() if p.requires_grad)
        
    def forward(self, x, targets=None):
        # x is of shape (B, T), each element is an integer in the range [0, vocab_size) representing a token. By seeing each token as a one-hot vector, x can be seen as a tensor of shape (B, T, vocab_size)
        B, T = x.shape
        
        # Embedd x into the d_model dimension which makes x of shape (B, T, d_model)
        x = self.embedding(x)
        
        # MLP layer into an unembedding layer, which makes x of shape (B, T, vocab_size)
        x = self.linear(x)
        
        # Softmax over the vocabulary dimension which is the last dimension, returns a probability distribution over the vocabulary, it is of shape (B, T, vocab_size)
        logits = F.softmax(x, dim=-1)
        
        if targets is not None:
            # Calculate the loss
            
            # Flatten the logits and targets to be of shape (B*T, vocab_size) and (B*T) respectively
            x = x.view(B*T, self.vocab_size)
            targets = targets.view(B*T)
            
            loss = F.cross_entropy(logits, targets)
            return loss, logits
        
        return logits

In [19]:
class positional_encoding(nn.Module):
    """ 
    Positional encoding according to [VSP17] paper "Attention is all you need" based on sine and cosine functions.
    
    Input: x a sequence of tokens of shape (B, T, d_model)
    Output: p, where p is the positional encoding, of shape (B, T, d_model)
    """
    def __init__(self, config=MASTER_CONFIG):
        super().__init__()
        d_model = config.d_model
        l_max = config.l_max
        dtype = config.dtype
        
        self.p = torch.zeros((1, l_max, d_model)) #TODO: Is this really necessary? Should this be l_max, d_model instead?
        
        # Creates X = [[0], [1], ..., [l_max - 1]]
        num = torch.arange(l_max, dtype=dtype).reshape(-1, 1)
        # Creates Y = [10000^0/d_model, 10000^2/d_model, ..., 10000^(d_model - 1)/d_model]
        denum = torch.pow(10000, torch.arange(0, d_model, 2, dtype=dtype) / d_model)
        
        self.p[:, :, 0::2] = torch.sin(num / denum)
        self.p[:, :, 1::2] = torch.cos(num / denum)
        
    def forward(self, x):
        return self.p[:, :x.shape[1], :].to(x.device)

class AttentionHead(nn.Module):
    def __init__(self, config=MASTER_CONFIG):
        super().__init__()
        self.config = config
        d_model = config.d_model
        
        # Linear layers for Q, K, V of dimensions dq, dk, dv respectively
        # TODO: More efficient to do the linear transformation together and then split the result?
        # TODO: Allow a toggle for whether to use bias or not
        self.linear_q = nn.Linear(d_model, d_model) 
        self.linear_k = nn.Linear(d_model, d_model)  
        self.linear_v = nn.Linear(d_model, d_model)
        
        # Y_t = att(X_t W_h^Q, X_t W_h^K, X_t W_h^V) = softmax((X_t W_h^Q)(X_t W_h^K)^t / sqrt(d_model)) (X_t W_h^V)
        
        # Linear layer for the output of the attention mechanism
        self.linear_o = nn.Linear(d_model, d_model)
        
    def forward(self, x, mask=None):
        # x is of shape (B, T, d_model)
        B, T, D = x.shape
        
        q = self.linear_q(x) # (B, T, d_model)
        k = self.linear_k(x) # (B, T, d_model)
        v = self.linear_v(x) # (B, T, d_model)
        
        # (B, T, d_model) * (B, d_model, T) = (B, T, T)
        if mask == None:
            weights = torch.bmm(q, k.transpose(1, 2)) / np.sqrt(D)
        else:
            # TODO: Implement masking
            pass

        # Apply the softmax on the last dimension, meaning the last dimension sums to 1
        weights = F.softmax(weights, dim=-1)
        
        # Apply the attention weights to the values
        # (B, T, T) * (B, T, d_model) = (B, T, d_model)
        v_bar = torch.bmm(weights, v)
        
        return v_bar
        

class MultiHeadAttention(nn.Module):
    def __init__(self, config=MASTER_CONFIG):
        super().__init__()
        self.config = config
        
        d_model = config.d_model
        n_heads = config.n_heads
        
        assert d_model % n_heads == 0, "d_model must be divisible by n_heads"
        
        head_size = d_model // n_heads
        
        self.heads = nn.ModuleList(
            [AttentionHead(config) for _ in range(n_heads)]
        )
        
        self.linear_o = nn.Linear(head_size * n_heads, d_model)
        
    def forward(self, x):
        # x is of shape (B, T, d_model)
        B, T, D = x.shape
        H = self.config.n_heads
        
        # Reshape x to (B, T, H, W)
        x = x.view(B, T, H, -1)
        
        # Transpose to get shape (B, H, T, W)
        x = x.transpose(1, 2)
        
        # Apply attention heads to shape (B, H, T, W)
        v = [head(x[:, i, :, :]) for i, head in enumerate(self.heads)] 
        # TODO: Something wrong here with tensor dimensions
        
        # Stack and reshape
        v = torch.stack(v, dim=1)
        v = v.transpose(1, 2).contiguous().view(B, T, -1)
        
        # Apply the linear layer
        v_bar = self.linear_o(v)
        
        return v_bar
    
class MultiHeadAttentionLayer(nn.Module):
    def __init__(self, config=MASTER_CONFIG):
        super().__init__()
        self.config = config
        
        self.ln_mha = nn.LayerNorm(config.d_model)
        self.mha = MultiHeadAttention(config)
        
        self.fcn = nn.Sequential(
            nn.LayerNorm(config.d_model),
            nn.Linear(config.d_model, config.d_model),
            nn.GELU(),
            nn.Linear(config.d_model, config.d_model)
        )
        
    def forward(self, x):
        # x is of shape (B, T, d_model)
        
        x = self.ln_mha(x)
        x = self.mha(x) + x
        x = self.fcn(x) + x
        
        return x

class cup_GPT(nn.Module):
    def __init__(self, config=MASTER_CONFIG):
        super().__init__()
        self.config = config
        
        self.embed = nn.Embedding(config.vocab_size, config.d_model)
        self.pos_enc = positional_encoding(config)
        
        self.mha_layers = nn.ModuleList(
            [MultiHeadAttentionLayer(config) for _ in range(config.n_layers)]
        )
        
        self.ln = nn.LayerNorm(config.d_model)
        self.unembed = nn.Linear(config.d_model, config.vocab_size)
        
    def forward(self, x):
        # x is of shape (B, T)
        B, T = x.shape
        
        # Embed the tokens (B, T) -> (B, T, d_model)
        x = self.embed(x)
        
        # Add the positional encoding (B, T, d_model) -> (B, T, d_model)
        x = x + self.pos_enc(x)
        
        # Apply the MHA layers (B, T, d_model) -> (B, T, d_model)
        for mha_layer in self.mha_layers:
            x = mha_layer(x)
        
        # Apply the layer norm (B, T, d_model) -> (B, T, d_model)
        x = self.ln(x)
        
        # Apply the unembedding layer (B, T, d_model) -> (B, T, vocab_size)
        x = self.unembed(x)
        
        return x
    

# TODO: Next steps are to implement the decoder part of the transformer.
# Implement:
# - Attention mechanism
# - Multi-head attention
# - Layer norm
# - Feed forward network
# - Blocks
# - Training loop
# TODO: residual pass implementation

# Training pass

In [20]:
# Training of DummyModel
def train(model, optimizer, config=MASTER_CONFIG):
    # Set the model to training mode
    model.train()
    
    # Generate a batch of data
    inputs, targets = generate_batch("train", config)
    
    # Forward pass
    loss, logits = model(inputs, targets)
    
    # Backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    return loss.item()

# Initializing dummy model

In [30]:
config = MASTER_CONFIG
data = generate_cup_data(config.n_cups, config.n_moves, config.n_samples)

model = DummyModel(config)
#model.to(config.device)

optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate)

print(MASTER_CONFIG.vocab_size)

# Test model
x = torch.randint(MASTER_CONFIG.vocab_size, (1, 3))
y = model(x)

print(x, y)

20
tensor([[11,  7, 19]]) tensor([[[0.0532, 0.0408, 0.0340, 0.0502, 0.0486, 0.0626, 0.0407, 0.0346,
          0.0560, 0.0509, 0.0487, 0.0771, 0.0573, 0.0519, 0.0736, 0.0322,
          0.0399, 0.0469, 0.0320, 0.0688],
         [0.0432, 0.0496, 0.0334, 0.0584, 0.0421, 0.0424, 0.0454, 0.0542,
          0.0681, 0.0495, 0.0528, 0.0565, 0.0528, 0.0433, 0.0530, 0.0358,
          0.0627, 0.0534, 0.0542, 0.0492],
         [0.0487, 0.0634, 0.0366, 0.0467, 0.0568, 0.0563, 0.0360, 0.0572,
          0.0584, 0.0482, 0.0527, 0.0397, 0.0329, 0.0602, 0.0550, 0.0494,
          0.0542, 0.0473, 0.0347, 0.0654]]], grad_fn=<SoftmaxBackward0>)


In [22]:
config = MASTER_CONFIG
data = generate_cup_data(config.n_cups, config.n_moves, config.n_samples)

model = cup_GPT(config)
#model.to(config.device)

optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate)

print(MASTER_CONFIG.vocab_size)

# Test model
x = torch.randint(MASTER_CONFIG.vocab_size, (1, 3))
y = model(x)

print(x, y)

20


RuntimeError: mat1 and mat2 shapes cannot be multiplied (3x16 and 128x128)

# Tests

In [ ]:
def test_auto_regressive(model, config=MASTER_CONFIG):
    '''
    Tests if the model is auto-regressive by comparing the output of the model when given the entire input sequence and when given the input sequence one token at a time.
    '''
    
    # shape vocab_size, batch_size, block_size
    x = torch.randint(config.vocab_size, (1, 3))
    y1 = model(x)
    
    y2 = torch.zeros_like(y1)
    for b in range(x.size(1)):
        y_b = model(x[:, :b + 1])
        y2[:, b] = y_b[:, b]
            
    error = ((y1 - y2).norm() / (y1.norm() + y2.norm())).item()
    
    if error < 1e-5:
        print("Auto-regressive test passed")
    else:
        print("Auto-regressive test failed")
        
    # print(f"Error={error}")
    
test_auto_regressive(model)

Auto-regressive test passed


In [ ]:
# Method for evaluating the loss of the PyTorch model on the validation set without defining the model.
@torch.no_grad()
def evaluate_model(model, criterion):
    out = {}
    model.eval()
    
    for split in ["train", "val"]:
        total_loss = 0
        total_tokens = 0
        
        for batch in generate_batch(split=split):
            # Get the inputs and targets
            inputs = batch["inputs"]
            targets = batch["targets"]
            
            # Get the model outputs
            outputs = model(inputs)
            
            # Calculate the loss
            loss = criterion(outputs, targets)
            
            # Update the total loss and tokens
            total_loss += loss.item()
            total_tokens += targets.shape[0] * targets.shape[1]
        
        # Calculate the average loss
        avg_loss = total_loss / total_tokens
        
        # Store the average loss
        out[f"{split}_loss"] = avg_loss
    

# Miscelanea

In [ ]:
def positional_embedding_naive(t: int, config=MASTER_CONFIG) -> torch.Tensor:
    """
    Input: t the position of the token in a sequence
    Output: the positional embedding of the position
    Parameters: W_p in R^{d_model x l_max} where l_max is the maximum sequence length or context size
    Return: e_p in R^{d_model}

    Using the sine positional embedding from [VSP17] paper "Attention is all you need".
    """
    d_model = config.d_model
    l_max = config.l_max
    
    # Create the positional embedding matrix
    W_p = torch.zeros((d_model, l_max))
    for t in range(l_max):
        for i in range(d_model):
            if i % 2 == 0:
                W_p[i, t] = np.sin(t / 10000 ** (i / d_model)) # TODO: check if this is correct, or should it be l_max instead of 10000?
            else:
                W_p[i, t] = np.cos(t / 10000 ** ((i - i % 2) / d_model))
    
    # Return the positional embedding vector
    return W_p[:, t]
